In [16]:
# Growth Area Coordinates
import pandas as pd
import requests
import json
import hvplot.pandas
from config import geoapify_key

In [17]:
growth_data = pd.read_csv("Output/grmelbourne_growth_byarea.csv")
ptv_data = pd.read_csv("Output/PTV_suburb.csv")
suburb_list = pd.DataFrame (growth_data[["Area Name"]])
suburb_list["Lat"] = ""
suburb_list["Lon"] = ""

# suburb_list=suburb_list.sample(10)


In [18]:
base_url = "https://api.geoapify.com/v1/geocode/search"
params = {
    "apiKey":geoapify_key,
    "format":"json"
    # "filter":"rect:144.16081518870078,-37.3613122327737,146.47573481130252,-38.5354962411197"
}

for index, row in suburb_list.iterrows():
    area = row["Area Name"] + ",Vic,Australia"
    params["text"] = f"{area}"
    response = requests.get(base_url, params=params).json()
    # response = response.json()
    suburb_list.loc[index, "Lat"] = response["results"][0]["lat"]
    suburb_list.loc[index, "Lon"] = response["results"][0]["lon"]
    print(f"Coordinates for {area} fetched...")
suburb_list.head()

Coordinates for Brunswick,Vic,Australia fetched...
Coordinates for Brunswick East,Vic,Australia fetched...
Coordinates for Brunswick West,Vic,Australia fetched...
Coordinates for Coburg,Vic,Australia fetched...
Coordinates for Pascoe Vale South,Vic,Australia fetched...
Coordinates for Alphington - Fairfield,Vic,Australia fetched...
Coordinates for Northcote,Vic,Australia fetched...
Coordinates for Thornbury,Vic,Australia fetched...
Coordinates for Ascot Vale,Vic,Australia fetched...
Coordinates for Essendon,Vic,Australia fetched...
Coordinates for Flemington,Vic,Australia fetched...
Coordinates for Moonee Ponds,Vic,Australia fetched...
Coordinates for Carlton,Vic,Australia fetched...
Coordinates for Docklands,Vic,Australia fetched...
Coordinates for East Melbourne,Vic,Australia fetched...
Coordinates for Flemington Racecourse,Vic,Australia fetched...
Coordinates for Kensington,Vic,Australia fetched...
Coordinates for West Melbourne,Vic,Australia fetched...
Coordinates for North Melbour